# 📖 Introduction

This is a notebook that summarizes some of the knowledge gained through the **5-day Gen AI Intensive Course with Google** in a form of the **Gen AI Intensive Course Capstone 2025Q1**.  

## It is a refactored approach to use **LangChain** capabilities where possible.

It implements the intelligent chef assistant bot, whose main capabilities are:
* selection of proper cookbook based on users suggestion
* suggestion of a recipe eg. based on available ingredients
* dummy ordering of ingredients

The **gen AI capabilities** used in the notebook are:  
✅ Embeddings  
✅ Few shot prompting  
✅ Structured output/JSON mode/controlled generation  
✅ Retrieval augmented generation (RAG)  
✅ Vector search/vector store/vector database   
✅ Agents with LangGraph

# ⚒ Installation and setup

In [1]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai

!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7' 'langchain' 'langchain-community'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 10.8 MB/s eta 0:00:00


In [2]:
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.5 MB/s eta 0:

Verify installed genai version

In [3]:
from google import genai
from google.genai import types

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.__version__

'1.7.0'

Setup the API key and env variable.

In [4]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# This is crucial, necessary for LangGraph invoke
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [5]:
from enum import Enum, auto

class LLMProvider(Enum):
    GOOGLE = auto()

OUR_LLM_PROVIDER = LLMProvider.GOOGLE

In [6]:
from abc import ABC, abstractmethod

class LLMInterface(ABC):

    @abstractmethod
    def get_gen_llm():
        pass
    
    @abstractmethod
    def get_gen_llm():
        pass
    
    @abstractmethod
    def get_gen_llm():
        pass

    @abstractmethod
    def get_embed_kwarg():
        pass



In [7]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

class GoogleLLM(LLMInterface):
    def get_gen_llm(self):
        kwargs = dict(model="gemini-1.5-flash")
        return GoogleGenerativeAI(**kwargs)
     
    def get_embed_llm(self):
        kwargs = dict(model="models/text-embedding-004")
        return GoogleGenerativeAIEmbeddings(**kwargs)

    def get_chat_llm(self):
        kwargs = dict(model="gemini-2.0-flash")
        return ChatGoogleGenerativeAI(**kwargs)

    def get_embed_kwarg(self, kwarg):
        return kwarg

In [8]:
def create_llm(llm_provider: LLMProvider):
    print(f"Your provider is {llm_provider.name}!")
    
    if llm_provider == LLMProvider.GOOGLE:
        return GoogleLLM()
        
    else:
        raise NotImplementedError(f"The provider {provider} is not supported!")


my_factory = create_llm(llm_provider=OUR_LLM_PROVIDER)

gen_model = my_factory.get_gen_llm()
embed_model = my_factory.get_embed_llm()
chat_model = my_factory.get_chat_llm()

Your provider is GOOGLE!


# 📚 Cookbook data corpus preparation

From the attached dataset **Cookbooks** select some books and get the first N characters, based on which the titles will be retrieved later.

In [9]:
import os
import re
import json
import typing_extensions as typing
from google.api_core import retry
from langchain_community.document_loaders import TextLoader

CLIP = 250
NUM_BOOKS = 5
BOOKS_STEP = 12

book_headers = []
book_file_names = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in sorted(filenames)[::BOOKS_STEP][:NUM_BOOKS]:

        book_loader = TextLoader(os.path.join(dirname, filename))
        book = book_loader.load()
        
        book_headers.append(book[0].page_content[:CLIP])
        book_file_names.append(filename)
        print(filename)

amem.txt
chin.txt
epia.txt
grea.txt
linc.txt


## 📝 Titles retrieval
Define Pydantic model for a function output, to help structure the few_shot_prompt and LLM call output format.
Capabilities:
* **few shot prompting**
* **structured output controlled generation**

In [10]:
from pydantic import BaseModel, Field

class BookInfo(BaseModel):
    title: str = Field(description="Title of a book")
    authors: list[str] = Field(description="Book authors list")

In [11]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.output_parsers import JsonOutputParser

few_shot_prompt_instruction = "Parse the begining of given book to retrieve title and authors. Note there can be many new-line characters inside the text."
examples = [{"input": "\n \n \n \n The American Woman's Home: or, Principles of Domestic Science; being a Guide to the Formation and Maintenance of Economical, Healthful, Beautiful, and Christian Homes.  Beecher, Catharine Esther  Stowe, Harriet Beecher  Home economics.  Introduction. The Christian Family. A Christian House. A Healthful Home.",
            "output": 
                """
                title: "The American Woman's Home: or, Principles of Domestic Science; being a Guide to the Formation and Maintenance of Economical, Healthful, Beautiful, and Christian Homes.\n"
                authors: ["Catharine Beecher", "Stowe Esther", "Beecher Harriet"]
                """,
            },           
            {"input": "\n\n Directions for Cookery, in its Various Branches.\n Leslie, Eliza \nCookery, American.\n",
            "output":
                """
                title: "Directions for Cookery, in its Various Branches."
                authors: ["Eliza Leslie"]
                """,
            },
            {"input": "\n\n \n\n \nA bookplate illustration of a illuminated reading lap and an open book.  \nThis book belongs to Beatrice V. Grant.\n\n",
            "output":
                """
                title: "A bookplate illustration of a illuminated reading lap and an open book."
                authors: ["Beatrice V. Grant"]
                """
           }]

example_prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = "EXAMPLE: {input}\nResponse: {output}",  
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=500,
)

output_parser = JsonOutputParser(pydantic_object=BookInfo)
format_instructions = output_parser.get_format_instructions()

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=few_shot_prompt_instruction + "\n {format_instructions}",
    suffix="EXAMPLE: {header}\nResponse:",
    input_variables=["header"],
    partial_variables={"format_instructions": format_instructions},
)

In [12]:
print(dynamic_prompt.format(header="Cookbook for oldies. Mr. Matuzalem"))

Parse the begining of given book to retrieve title and authors. Note there can be many new-line characters inside the text.
 The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"title": {"description": "Title of a book", "title": "Title", "type": "string"}, "authors": {"description": "Book authors list", "items": {"type": "string"}, "title": "Authors", "type": "array"}}, "required": ["title", "authors"]}
```

EXAMPLE: 
 
 
 
 The American Woman's Home: or, Principles of Domestic Science; being a Guide to the Formation and Maintenance of Economical, Healthful, Beautiful, and Chri

In [13]:

@retry.Retry(predicate=is_retriable, timeout=3.0)
def extract_header_meta(book_header: str) -> dict:

    chain = dynamic_prompt | gen_model | output_parser
    
    return chain.invoke({"header": book_header})
    

titles_retrieved = [] 
for book_header in book_headers:
    print("=========================")
    print("Book header:\n")
    print(book_header.replace("\n", ""))

    try:        
        book_info = extract_header_meta(book_header)
        titles_retrieved.append(book_info.get("title"))
        print("=========================")
        print("Retrieved title and authors:")
        print(book_info)
        print("")
    except:
        print("error")
        titles_retrieved.append("")
    print("=========================\n")

Book header:

         The American Matron: Or, Practical and Scientific Cookery.  By a Housekeeper.  Boston: J. Munroe &amp; Co., 1851  [Page images for  The American Matron  were produced before MSU began the "Feeding America" digitization pro
Retrieved title and authors:
{'title': 'The American Matron: Or, Practical and Scientific Cookery.', 'authors': ['A Housekeeper']}


Book header:

      Chinese-Japanese Cook Book  Bosse, Sara  Watanna, Onoto  Cookery, Chinese. Cookery, Japanese. Cookery, American.  Part 1 Chinese Recipes. Rules for Cooking. Soups. Gravy. Fish. Poultry and Game. Meats. Chop Sueys. Chow Mains. Fried Rice
Retrieved title and authors:
{'title': 'Chinese-Japanese Cook Book', 'authors': ['Sara Bosse', 'Onoto Watanna']}


Book header:

     The Epicurean...  Ranhofer, Charles.  Cookery, American. Cookery, French. Menus.  Complete title: The Epicurean. A complete treatise of Analytical and Practical Studies on the Culinary Art including Table and Wine Service, How to 

# 🧠 RAG utilities
## Text chunking

In [14]:
from langchain.text_splitter import TokenTextSplitter

splitter = TokenTextSplitter(
    encoding_name="cl100k_base", # Example encoding for newer OpenAI models
    chunk_size=500,  # Target chunk size in TOKENS
    chunk_overlap=30 # Overlap in TOKENS
)

chunks = splitter.split_documents(book)

# example:
print(chunks[0].page_content)

 
 
 
 
 Mrs. Lincoln's Boston Cook Book. What to Do and What Not to Do in Cooking. 
 Lincoln, Mary Johnson 
 Cookery, American. 
 Introduction. Bread and Bread Making. Receipts for Yeast and Bread. Raised Biscuit, Rolls, etc. Stale Bread, Toast, etc. Soda Biscuit, Muffins, Gems, etc. Waffles and Griddle-Cakes. Fried Muffins, Fritters, Doughnuts, etc. Oatmeal and other Grains. Beverages. Soup and Stock. Soup without Stock. Fish. Shell Fish. Meat and Fish Sauces. Eggs. Meat. Beef. Mutton and Lamb. Veal. Pork. Poultry and Game. Entr&#233;es and Meat R&#233;chauff&#233;. Sundries. Vegetables. Rice and Macaroni. Salads. Pastry and Pies. Pudding Sauces. Hot Puddings. Custards, Jellies, and Creams. Ice-Cream and Sherbet. Cake. Fruit. Cooking for Invalids. Miscellaneous Hints. The Dining-Room. The Care of Kitchen Utensils. An Outline of Study for Teachers. Suggestions to Teachers. A Course of Study for Normal Pupils. Miscellaneous Questions for Examination. Topics and Illustrations for Lectur

## Embedding function for RAG system
This will make embeddings of text chunks (obtained with dummy_chunk_text) that will be stored in a vector database. Later a user query will allow to retrieve (hopefully) the most relevant chunks.

In [15]:
from chromadb import Documents, EmbeddingFunction, Embeddings

class LangChainEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    embedding_task = "retrieval_document"

    def __init__(self, *args, embedding_llm, **kwargs):
        super().__init__(*args, **kwargs)
        self.embedding_llm = embedding_llm

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:

        kwargs = {"task_type": self.embedding_task} if self.embedding_task else {}        
        response = self.embedding_llm.embed_documents(input, **kwargs)
        return response

Helper method to navigate through book titles.

In [16]:
from IPython.display import Markdown

def truncate(t: str, limit: int = 50) -> str:
  """Truncate labels to fit on the chart."""
  if len(t) > limit:
    return t[:limit-3] + '...'
  else:
    return t

# 🔩 Tools for LLM to use
Several tools are specified here:
* find_cuisine - Find appropriate cookbook based on how the provided query matches the title of a book - based on semantic similarity (capability: **Embeddings**)
* summarize_cookbook - Summarize cookbook (perform indexing with vector database) - (capability: **RAG**)
* retrieve_recipe - Retrieves relevant information about user requested recipe - (capability: **RAG**)
* order_ingredients - Orders desired ingredients in the nearby store - (capability: just **function calling** like other methods)

In [17]:
import numpy as np
import chromadb
import pandas as pd

STEP = 100
DB_NAME = "db_local"
embed_fn = LangChainEmbeddingFunction(embedding_llm=embed_model)

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)


def find_cuisine(cuisine_query: str) -> str:
    """Find appropriate cookbook based on how the provided query matches the title of a book"""
    query = titles_retrieved + [cuisine_query]
    
    truncated_texts = [truncate(t) for t in query]

    embed_fn.embedding_task = "semantic_similarity"
    response = embed_fn(input=query)
    
    df = pd.DataFrame(response, index=truncated_texts)
    
    # Perform the similarity calculation
    sim = df @ df.T
    filename = book_file_names[sim[cuisine_query].iloc[:-1].argmax()]
    return filename


def summarize_cookbook(filename: str) -> None:
    """Summarize cookbook (perform indexing with vector database"""

    book_loader = TextLoader(os.path.join('/kaggle/input/cookbooks', filename))
    book = book_loader.load()
    chunks = [c.page_content for c in splitter.split_documents(book)]
    
    embed_fn.embedding_task = "retrieval_document"

    # Chunk the book as at most 100 vectors can be added in single batch to the database
    for x in range(0, len(chunks), STEP):
        small_chunk = chunks[x: x+STEP-1]
        db.add(documents=small_chunk, ids=[str(i + x) for i in range(len(small_chunk))])

def retrieve_recipe(recipe_query: str) -> str:
    """Retrieves relevant information about user requested recipe.

    Parameters:
    recipe_query: str
        user query about recipe he wants to get

    Returns:
        str: chunks of relevant recipes found in the cookbook.
    """
    # Switch to query mode when generating embeddings.
    embed_fn.embedding_task = "retrieval_query"
    
    result = db.query(query_texts=[recipe_query], n_results=5)
    [all_passages] = result["documents"]

    prompt = ""
    # Add the retrieved documents to the prompt.
    for passage in all_passages:
        passage_oneline = passage.replace("\n", " ")
        prompt += f"PASSAGE: {passage_oneline}\n"

    return prompt



def order_ingredients(ingredients: list[str]) -> bool:
    """Orders desired ingredients in the nearby store.
    
    Parameters:
    ingredients: tuple[str]
        iterable of ingredients to order

    Returns:
        bool: whether the order was succesfull
    
    """
    total_succes = 0
    for item in ingredients:
        if np.random.random()<0.9:
            print(f"{item} ordered!")
            total_succes += 1
        else:
            print(f"{item} is missing in the shop!")

    return total_succes == len(ingredients)

Prepare chat bot instructions.

In [18]:
chat_instructions = f"""You are a helpful bot - the cook assistant. 
You should guide the user through following steps:
- ask about the type of cuisine he wants - use tool find_cuisine for you to find the appropriate cookbook file_name
- now as you have access to a selected cookbook with many of interesting recipes - you can use a tool summarize_cookbook which needs the file_name found
- you should propose a recipe based on ingredients indicated by your chef. Your output should contain four parts:
* Name of dish and extremely brief summary
* Detailed preparation guide (try to describe each step based also on your knowledge - put this information that is not present in the book in the braces)
* Ingredients chef already has
* Ingredients missing and what is missing)

You have acess also to tools order_ingredients to order desired ingredients for a selected recipe AND a atool retrieve_recipe - in order to get recipe chunks from the cookbook based on which 
you summarize it according to a provided guide. Note that you have to always rely on the recipe text chunks obtained with the retrieve_recipe. You are not allowed to propose a recipe that does not come
from a retrieve_recipe text chunks.
Be eager to run tools by yourself, be verbose and summarize what you have done.

"""

In [19]:
bot_tools = [order_ingredients, retrieve_recipe, find_cuisine, summarize_cookbook]

# 🥷 Agentic approach 

## State class
Prepare class to store conversation history and success state of the conversation, along with ingredients lits.

In [20]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class RecipeState(TypedDict):
    """State representing the cook chef's conversation."""

    # chats history
    messages: Annotated[list, add_messages]

    # list of ingredients eventually to buy
    order: list[str]

    # Flag indicating that the chef is satisfied
    finished: bool

## Extended and updated instruction for the Agent

In [21]:
CHEF_BOT_SYSINT = (
    "system",
    chat_instructions + " If order of products was not succesfull, allow chef to continue without the product if he wishes to or try to order them again - ask chef for that.\n "
    "Say goodbye and wish buon appetite when finished. "
    "After some software update, all functions you know now have the suffix '_tool' added, so it is a 'find_cuisine_tool' rather than just 'find_cuisine' and 'retrieve_recipe_tool' rather than 'retrieve_recipe' and so on. "
    "Additionally you have confirm_satisfaction_tool to finish conversation when chef is satified and add_to_order_tool to place desired ingredients to order in the nearbye store.",
)
WELCOME_MSG = "Welcome to the ChefBot helper. Type `q` to quit. How may I serve you today?"

## Tools for the Agent and nodes for the graph

In [22]:
from langchain_core.messages.tool import ToolMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langchain_core.messages.ai import AIMessage
import time
from collections.abc import Iterable

@tool
def find_cuisine_tool(cuisine_query: str) -> str:
    """Finds the appropriate cookbook for users request."""
    return find_cuisine(cuisine_query)

@tool
def summarize_cookbook_tool(filename: str):
    """Summarizes the selected cookbook."""
    return summarize_cookbook(filename)

@tool
def retrieve_recipe_tool(recipe_query: str) -> str:
    """Retrieves recipes as RAG utility."""
    return retrieve_recipe(recipe_query)

@tool
def add_to_order_tool(ingredients: Iterable[str]) -> str:
    """Adds the specified ingredient for order.

    Returns:
      The updated order in progress.    """


@tool
def order_ingredients_tool() -> str:
    """Asks the chef if the order is correct.

    Returns:
      The user's free-text response.
    """


@tool
def confirm_satisfaction_tool():
    """Confirms chef's satisfaction with the help from chatbot, after recipe is presented and ingredients discussed.
    """



def actions_node(state: RecipeState) -> RecipeState:
    tool_msg = state.get("messages", [])[-1]
    order = state.get("order", [])
    outbound_msgs = []
    chat_finished = False

    for tool_call in tool_msg.tool_calls:
        if tool_call["name"] == "add_to_order_tool":            
            order.append(f'{tool_call["args"]["ingredients"]}')
            response = "Added to order: " + "\n".join(order)
        
        elif tool_call["name"] == "confirm_order_tool":
            print("Your order:")
            if not order:
                print("  (no items)")

            for ingredient in order:
                print(f"  {ingredient}")

            response = input("Is this correct? ")
        
        elif tool_call["name"] == "order_ingredients_tool":
            response = "Ingredients ordered!"
            
        elif tool_call["name"] == "confirm_satisfaction_tool":
            response = "I am glad I could help!"
            chat_finished = True
            
        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')

        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    
    return {"messages": outbound_msgs, "order": order, "finished": chat_finished}


def human_node(state: RecipeState) -> RecipeState:
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}


# Comment this out if you wish interactive chat (based on 'input') instead of pre arranged user messages (for notebook demonstration purposes)
input_strings = ["I want chinese food today.", 
                 "All I have is some rice, I will take your suggested recipe.", 
                 "Please, add to order missing ingredients and order them.", 
                 "I am happy with your help, thank you."]
input_iterator = iter(input_strings)

def human_node(state: RecipeState) -> RecipeState:
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    # Wait some time to decrease calls / minute to the API
    time.sleep(30) 
    
    try:
        user_input = next(input_iterator)  # Get the next input from the iterator
        print(f"User: {user_input}") # Print user input from array
    except StopIteration:
        state["finished"] = True
        return state  # Exit if the iterator is exhausted

    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}

## Conditional edges

In [23]:
from typing import Literal

def maybe_route_to_tools(state: RecipeState) -> str:
    """Route between chat and tool nodes if a tool call is made."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if state.get("finished", False):
        return END

    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        if any(
            tool["name"] in tool_node.tools_by_name.keys() for tool in msg.tool_calls
        ):
            return "context_tools"
        else:
            return "action_tools"
    
    else:
        return "human"


def maybe_exit_human_node(state: RecipeState) -> Literal["chatbot", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"

## Full graph building for the Agent

In [24]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

def chatbot_with_tools(state: RecipeState) -> RecipeState:
    defaults = {"order": [], "finished": False}

    if state['messages']:
        new_output = llm_with_tools.invoke([CHEF_BOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    return defaults | state | {'messages': [new_output]}


# --------- tools
auto_tools = [find_cuisine_tool, summarize_cookbook_tool, retrieve_recipe_tool]
actions_tools = [add_to_order_tool, order_ingredients_tool, confirm_satisfaction_tool]

tool_node = ToolNode(auto_tools)

llm_with_tools = llm.bind_tools(auto_tools + actions_tools)

# --------- graph
graph_builder = StateGraph(RecipeState)

# nodes
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("context_tools", tool_node)
graph_builder.add_node("action_tools", actions_node)

# edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("context_tools", "chatbot")
graph_builder.add_edge("action_tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
graph_builder.add_conditional_edges("human", maybe_exit_human_node)

chat_graph = graph_builder.compile()

## 💬 Test the Agentic Chatbot 
In this notebook a pre arranged user messages are run in a chat with the bot.

In [25]:
config = {"recursion_limit": 100}
state = chat_graph.invoke({"messages": []}, config)

Model: Welcome to the ChefBot helper. Type `q` to quit. How may I serve you today?
User: I want chinese food today.
Model: Okay, I have loaded the Chinese cookbook. Now, tell me which ingredients you already have, so I can suggest a recipe.
User: All I have is some rice, I will take your suggested recipe.
Model: Okay, I have a few recipes that use rice. How about "Fried Rice with Herbs"?

*Name of dish and extremely brief summary:* Fried Rice with Herbs - A simple and tasty fried rice dish with celery, onion, and water chestnuts.

*Detailed preparation guide:*

1.  Fry one large onion a light brown in one and one half tablespoonfuls of pork fat. (Make sure the onion is evenly browned to add a sweet and savory base to the dish.)
2.  Chop up three stalks of celery very fine, and add five water chestnuts, sliced thin.
3.  Fry all a light brown, then take two cups of rice that has boiled for twenty-five minutes, or use cold rice if you have any on hand. (Using cold rice is preferable as it

In [26]:
from pprint import pprint
pprint(state)
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")

{'finished': True,
 'messages': [AIMessage(content='Welcome to the ChefBot helper. Type `q` to quit. How may I serve you today?', additional_kwargs={}, response_metadata={}, id='7f4680aa-af42-4a69-bac1-6cd3a5a5841e'),
              HumanMessage(content='I want chinese food today.', additional_kwargs={}, response_metadata={}, id='6a3d42bc-992b-4397-9177-c53e4cb717ea'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'find_cuisine_tool', 'arguments': '{"cuisine_query": "chinese"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-1d42ec6a-7f5d-43b7-8c6f-0c333cbede3a-0', tool_calls=[{'name': 'find_cuisine_tool', 'args': {'cuisine_query': 'chinese'}, 'id': 'a888ea09-89b6-4283-83a4-86394fc02edd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 562, 'output_tokens': 9, 'total_tokens': 571, 'input_token_details': {'cache_read': 0}}